### Running Bayesian Parameter Estimation

This notebook is used to run the Bayesian parameter estimation methods for updating duration-based parameters used in post-earthquake recovery models. The code was written by Morolake Omoya and Henry Burton and the calculations are based on the following paper: Omoya, M, Burton, H. and Baroud, H. (2022). Bayesian parameter estimation of duration-based variables used in post-earthquake building recovery modeling, Earthquake Spectra, DOI: 10.1177/87552930211073717

Four additional files are needed to run this script. Three of them are .csv files (permitTime.csv, repairTime.csv and recoveryTime.csv) and the fourth is the bayesianParameterEstimation.py file which contains the class and methods that support the main script.

In [1]:
# Import relevant  packages
import numpy as np
import pandas as pd
import seaborn as sns
import math
import pathlib
import os
from scipy.stats import gamma
from scipy.stats import expon
from time import time
from bayesianParameterEstimation import BayesianParameterEstimation

### Loading Data Files
The cell below loads the following files: (i) permitTime.csv, (2) repairTime.csv and (3) recoveryTime.csv, which contain the data used to perform the Bayesian updating.

In [2]:
# Define base directory
baseDirectory = pathlib.Path(os.getcwd())

# Go to base directory
os.chdir(baseDirectory)

# Permit time
with open('permitTime.csv', 'r') as csvfile:
    permitTime = pd.read_csv(csvfile)
permitTime.head(5) 

# Import repair time data
with open('repairTime.csv', 'r') as csvfile:
    repairTime = pd.read_csv(csvfile)
#repairTime.head(5)     

# Import recovery time data
with open('recoveryTime.csv', 'r') as csvfile:
    recoveryTime = pd.read_csv(csvfile)
#recoveryTime.head(5) 

### Compute Duration Variable Statistics from Data
The cell below computes the statistics for the observed values of the duration variables.


In [3]:
# Extract data
permitTimeData = [permitTime.Green[np.isfinite(permitTime.Green)],permitTime.Yellow[np.isfinite(permitTime.Yellow)],permitTime.Red[np.isfinite(permitTime.Red)]]
repairTimeData = [repairTime.Slight[np.isfinite(repairTime.Slight)],repairTime.Moderate[np.isfinite(repairTime.Moderate)],repairTime.Extensive[np.isfinite(repairTime.Extensive)],repairTime.Complete[np.isfinite(repairTime.Complete)]]
recoveryTimeData = [recoveryTime.Slight[np.isfinite(recoveryTime.Slight)],recoveryTime.Moderate[np.isfinite(recoveryTime.Moderate)],recoveryTime.Extensive[np.isfinite(recoveryTime.Extensive)],recoveryTime.Complete[np.isfinite(recoveryTime.Complete)]]

# Initialize lists used to store statistics
nObsvPermitTime, meanObsvPermitTime, sdObsvPermitTime, lambdaObsvPermitTime = [],[],[],[] 
nObsvRepairTime, meanObsvRepairTime, sdObsvRepairTime, lambdaObsvRepairTime = [],[],[],[] 
nObsvRecoveryTime, meanObsvRecoveryTime, sdObsvRecoveryTime, lambdaObsvRecoveryTime = [],[],[],[] 

# Extract statistics
for i in range(4):
    if i < 3:
        # Extract permit time statistics        
        nObsvPermitTime.append(len(permitTimeData[i]))
        meanObsvPermitTime.append(np.mean(permitTimeData[i]))
        sdObsvPermitTime.append(np.std(permitTimeData[i]))
        lambdaObsvPermitTime.append(1/meanObsvPermitTime[i])
    
    # Extract repair time statistics        
    nObsvRepairTime.append(len(repairTimeData[i]))
    meanObsvRepairTime.append(np.mean(repairTimeData[i]))
    sdObsvRepairTime.append(np.std(repairTimeData[i]))
    lambdaObsvRepairTime.append(1/meanObsvRepairTime[i])
        
    # Extract recovery time statistics        
    nObsvRecoveryTime.append(len(recoveryTimeData[i]))
    meanObsvRecoveryTime.append(np.mean(recoveryTimeData[i]))
    sdObsvRecoveryTime.append(np.std(recoveryTimeData[i]))
    lambdaObsvRecoveryTime.append(1/meanObsvRecoveryTime[i])
        
# Print permit time statistics
print('nObsvPermitTime =' + str(nObsvPermitTime))
print("")
print('meanObsvPermitTime =' + str(meanObsvPermitTime))
print("")
print('sdObsvPermitTime =' + str(sdObsvPermitTime))
print("")
print('lambdaObsvPermitTime =' + str(lambdaObsvPermitTime))
print("")

# Print repair time statistics
print('nObsvRepairTime =' + str(nObsvRepairTime))
print("")
print('meanObsvRepairTime =' + str(meanObsvRepairTime))
print("")
print('sdObsvRepairTime =' + str(sdObsvRepairTime))
print("")
print('lambdaObsvRepairTime =' + str(lambdaObsvRepairTime))
print("")

# Print reocvery time statistics
print('nObsvRecoveryTime =' + str(nObsvRecoveryTime))
print("")
print('meanObsvRecoveryTime =' + str(meanObsvRecoveryTime))
print("")
print('sdObsvRecoveryTime =' + str(sdObsvRecoveryTime))
print("")
print('lambdaObsvRecoveryTime =' + str(lambdaObsvRecoveryTime))



nObsvPermitTime =[74, 728, 78]

meanObsvPermitTime =[109.95945945945945, 91.36813186813187, 112.83333333333333]

sdObsvPermitTime =[89.96026738408419, 103.59430155222104, 116.50557559747583]

lambdaObsvPermitTime =[0.009094260784072754, 0.010944735101329003, 0.008862629246676515]

nObsvRepairTime =[212, 366, 59, 35]

meanObsvRepairTime =[88.22641509433963, 126.45081967213115, 127.1864406779661, 186.45714285714286]

sdObsvRepairTime =[102.15621436031908, 173.1076816297268, 149.4769949508439, 172.83275861068583]

lambdaObsvRepairTime =[0.011334473909324207, 0.007908212873533415, 0.007862473347547975, 0.005363162733680662]

nObsvRecoveryTime =[173, 319, 55, 29]

meanObsvRecoveryTime =[175.79190751445086, 232.94043887147336, 233.05454545454546, 345.62068965517244]

sdObsvRecoveryTime =[127.31723858484389, 196.44543355521824, 159.349520714893, 179.08048928038946]

lambdaObsvRecoveryTime =[0.005688543995791135, 0.004292942886364859, 0.004290841004836948, 0.0028933453057966674]


### Define Parameters for Bayesian Parameter Estimation
The cell below defines the variables used in the Bayesian updating including the prior distribution parameters, the relevant data, and the number of samples (only applies to MCMC). For illustration purposes, we will update the mean permit time (and corresponding lambda) for the green-tagged buildings.

In [4]:
# Define prior distribution parameters
meanPriorMeanPermitTime = [55.6,149.9,368.4]
meanPriorMeanRepairTime = [2.4,35.9,107.7,215.5]
meanPriorMeanRecoveryTime = [6,143.7,431,862]

# Coefficient of variation
coefficientOfVariation = 0.3

# Specify the number of samples/iterations used in MCMC
NSamples = 100000

### Compute Prior Gamma Distribution Parameters
The cell below computes the $\alpha$ and $\beta$ values that define the prior (gamma) distribution

In [5]:
# Initialize lists used to store the gamma distribution parameters for the prior
lambdaPriorAlphaPermitTime,lambdaPriorAlphaRepairTime,lambdaPriorAlphaRecoveryTime = [],[],[]
lambdaPriorBetaPermitTime,lambdaPriorBetaRepairTime,lambdaPriorBetaRecoveryTime = [],[],[]

# Loop over the number of parameters
for i in range(4):
    if i < 3:
        # Compute gamma distribution parameters for permit time
        lambdaPriorAlphaPermitTime.append(((1/meanPriorMeanPermitTime[i])/(coefficientOfVariation/meanPriorMeanPermitTime[i]))**2)
        lambdaPriorBetaPermitTime.append(lambdaPriorAlphaPermitTime[i]/(1/meanPriorMeanPermitTime[i]))
    # Compute gamma distribution parameters for repair time
    lambdaPriorAlphaRepairTime.append(((1/meanPriorMeanRepairTime[i])/(coefficientOfVariation/meanPriorMeanRepairTime[i]))**2)
    lambdaPriorBetaRepairTime.append(lambdaPriorAlphaRepairTime[i]/(1/meanPriorMeanRepairTime[i]))
                                     
    # Compute gamma distribution parameters for recovery time
    lambdaPriorAlphaRecoveryTime.append(((1/meanPriorMeanRecoveryTime[i])/(coefficientOfVariation/meanPriorMeanRecoveryTime[i]))**2)
    lambdaPriorBetaRecoveryTime.append(lambdaPriorAlphaRecoveryTime[i]/(1/meanPriorMeanRecoveryTime[i]))
                                
# Print permit time gamma distribution parameters
print('lambdaPriorAlphaPermitTime =' + str(lambdaPriorAlphaPermitTime))
print("")
print('lambdaPriorBetaPermitTime =' + str(lambdaPriorBetaPermitTime))
print("")

# Print repair time gamma distribution parameters
print('lambdaPriorAlphaRepairTime =' + str(lambdaPriorAlphaRepairTime))
print("")
print('lambdaPriorBetaRepairTime =' + str(lambdaPriorBetaRepairTime))
print("")

# Print reocvery time gamma distribution parameters
print('lambdaPriorAlphaRecoveryTime =' + str(lambdaPriorAlphaRecoveryTime))
print("")
print('lambdaPriorBetaRecoveryTime =' + str(lambdaPriorBetaRecoveryTime))



lambdaPriorAlphaPermitTime =[11.111111111111112, 11.111111111111112, 11.111111111111112]

lambdaPriorBetaPermitTime =[617.7777777777778, 1665.5555555555559, 4093.3333333333335]

lambdaPriorAlphaRepairTime =[11.111111111111112, 11.111111111111112, 11.111111111111109, 11.111111111111112]

lambdaPriorBetaRepairTime =[26.666666666666668, 398.8888888888889, 1196.6666666666665, 2394.444444444445]

lambdaPriorAlphaRecoveryTime =[11.111111111111112, 11.111111111111109, 11.111111111111112, 11.111111111111112]

lambdaPriorBetaRecoveryTime =[66.66666666666669, 1596.6666666666663, 4788.88888888889, 9577.77777777778]


### Conjugate Prior Method
The cell below performs the Bayesian parameter estimation using the conjugate prior (CP) method.

In [6]:
# Start time check
t0 = time()

# Initialize lists used to store CP results
meanPosteriorMeanPermitTimeCP,lambdaPosteriorMeanPermitTimeCP,posteriorCOVPermitTimeCP,lambdaPosteriorAlphaPermitTimeCP,lambdaPosteriorBetaPermitTimeCP = [],[],[],[],[]
meanPosteriorMeanRepairTimeCP,lambdaPosteriorMeanRepairTimeCP,posteriorCOVRepairTimeCP,lambdaPosteriorAlphaRepairTimeCP,lambdaPosteriorBetaRepairTimeCP = [],[],[],[],[]
meanPosteriorMeanRecoveryTimeCP,lambdaPosteriorMeanRecoveryTimeCP,posteriorCOVRecoveryTimeCP,lambdaPosteriorAlphaRecoveryTimeCP,lambdaPosteriorBetaRecoveryTimeCP = [],[],[],[],[]

# Perform Bayesian Parameter Estimation using Conjugate Prior
for i in range(4):
    if i < 3:
        # Perform Bayesian parameter estimation for permit time
        bayesianParameterEstimation = BayesianParameterEstimation(permitTimeData[i],NSamples,1/meanPriorMeanPermitTime[i],coefficientOfVariation)
        bayesianParameterEstimation.cpEstimation()
        lambdaPosteriorMeanPermitTimeCP.append(bayesianParameterEstimation.posteriorCP['lambdaMean'])
        meanPosteriorMeanPermitTimeCP.append(bayesianParameterEstimation.posteriorCP['meanMean'])
        posteriorCOVPermitTimeCP.append(bayesianParameterEstimation.posteriorCP['COV'])
        lambdaPosteriorAlphaPermitTimeCP.append(bayesianParameterEstimation.posteriorCP['alpha'])
        lambdaPosteriorBetaPermitTimeCP.append(bayesianParameterEstimation.posteriorCP['beta'])
    
    # Perform Bayesian parameter estimation for repair time
    bayesianParameterEstimation = BayesianParameterEstimation(repairTimeData[i],NSamples,1/meanPriorMeanRepairTime[i],coefficientOfVariation)
    bayesianParameterEstimation.cpEstimation()
    lambdaPosteriorMeanRepairTimeCP.append(bayesianParameterEstimation.posteriorCP['lambdaMean'])
    meanPosteriorMeanRepairTimeCP.append(bayesianParameterEstimation.posteriorCP['meanMean'])
    posteriorCOVRepairTimeCP.append(bayesianParameterEstimation.posteriorCP['COV'])
    lambdaPosteriorAlphaRepairTimeCP.append(bayesianParameterEstimation.posteriorCP['alpha'])
    lambdaPosteriorBetaRepairTimeCP.append(bayesianParameterEstimation.posteriorCP['beta'])
    
    # Perform Bayesian parameter estimation for recovery time
    bayesianParameterEstimation = BayesianParameterEstimation(recoveryTimeData[i],NSamples,1/meanPriorMeanRecoveryTime[i],coefficientOfVariation)
    bayesianParameterEstimation.cpEstimation()
    lambdaPosteriorMeanRecoveryTimeCP.append(bayesianParameterEstimation.posteriorCP['lambdaMean'])
    meanPosteriorMeanRecoveryTimeCP.append(bayesianParameterEstimation.posteriorCP['meanMean'])
    posteriorCOVRecoveryTimeCP.append(bayesianParameterEstimation.posteriorCP['COV'])
    lambdaPosteriorAlphaRecoveryTimeCP.append(bayesianParameterEstimation.posteriorCP['alpha'])
    lambdaPosteriorBetaRecoveryTimeCP.append(bayesianParameterEstimation.posteriorCP['beta'])
t1 = time()

### Print Results from Conjugate Prior Method
The cell below prints the results from the Bayesian parameter estimation using the conjugate prior method.

In [7]:
# Print permit time results
print('lambdaPosteriorMeanPermitTimeCP =' + str(lambdaPosteriorMeanPermitTimeCP))
print("")
print('meanPosteriorMeanPermitTimeCP =' + str(meanPosteriorMeanPermitTimeCP))
print("")
print('posteriorCOVPermitTimeCP =' + str(posteriorCOVPermitTimeCP))
print("")
print('lambdaPosteriorAlphaPermitTimeCP =' + str(lambdaPosteriorAlphaPermitTimeCP))
print("")
print('lambdaPosteriorBetaPermitTimeCP =' + str(lambdaPosteriorBetaPermitTimeCP))
print("")

# Print repair time results
print('lambdaPosteriorMeanRepairTimeCP =' + str(lambdaPosteriorMeanRepairTimeCP))
print("")
print('meanPosteriorMeanRepairTimeCP =' + str(meanPosteriorMeanRepairTimeCP))
print("")
print('posteriorCOVRepairTimeCP =' + str(posteriorCOVRepairTimeCP))
print("")
print('lambdaPosteriorAlphaRepairTimeCP =' + str(lambdaPosteriorAlphaRepairTimeCP))
print("")
print('lambdaPosteriorBetaRepairTimeCP =' + str(lambdaPosteriorBetaRepairTimeCP))
print("")

# Print reocvery time statistics
print('lambdaPosteriorMeanRecoveryTimeCP =' + str(lambdaPosteriorMeanRecoveryTimeCP))
print("")
print('meanPosteriorMeanRecoveryTimeCP =' + str(meanPosteriorMeanRecoveryTimeCP))
print("")
print('posteriorCOVRecoveryTimeCP =' + str(posteriorCOVRecoveryTimeCP))
print("")
print('posteriorCOVRecoveryTimeCP =' + str(posteriorCOVRecoveryTimeCP))
print("")
print('lambdaPosteriorAlphaRecoveryTimeCP =' + str(lambdaPosteriorAlphaRecoveryTimeCP))
print("")
print('lambdaPosteriorBetaRecoveryTimeCP =' + str(lambdaPosteriorBetaRecoveryTimeCP))
print("")

# Print run time
print('Run time = ' + str(t1 - t0))

lambdaPosteriorMeanPermitTimeCP =[0.009721675783381773, 0.010840338051672494, 0.006910873855009522]

meanPosteriorMeanPermitTimeCP =[102.86292428198432, 92.2480457005412, 144.6995012468828]

posteriorCOVPermitTimeCP =[0.10839440602948862, 0.036782829532357814, 0.10593368273196731]

lambdaPosteriorAlphaPermitTimeCP =[85.11111111111111, 739.1111111111111, 89.11111111111111]

lambdaPosteriorBetaPermitTimeCP =[8754.777777777777, 68181.55555555556, 12894.333333333334]

lambdaPosteriorMeanRepairTimeCP =[0.011911541381928625, 0.008078663426314924, 0.008058130922279262, 0.005169149519206816]

meanPosteriorMeanRepairTimeCP =[83.95219123505976, 123.78285209192694, 124.09825673534075, 193.45542168674697]

posteriorCOVRepairTimeCP =[0.06694827640161777, 0.0514950323993972, 0.11942811429870495, 0.14726420810214477]

lambdaPosteriorAlphaRepairTimeCP =[223.11111111111111, 377.1111111111111, 70.11111111111111, 46.111111111111114]

lambdaPosteriorBetaRepairTimeCP =[18730.666666666668, 46679.88888888889

### Markov Chain Monte Carlo (MCMC) Method
The cell below performs the Bayesian parameter estimation using the Markov Chain Monte Carlo (MCMC) method.

In [8]:
# Start time check
t0 = time()

# Initialize lists used to store MCMC results
meanPosteriorMeanPermitTimeMCMC,lambdaPosteriorMeanPermitTimeMCMC,posteriorCOVPermitTimeMCMC,lambdaPosteriorAlphaPermitTimeMCMC,lambdaPosteriorBetaPermitTimeMCMC = [],[],[],[],[]
meanPosteriorMeanRepairTimeMCMC,lambdaPosteriorMeanRepairTimeMCMC,posteriorCOVRepairTimeMCMC,lambdaPosteriorAlphaRepairTimeMCMC,lambdaPosteriorBetaRepairTimeMCMC = [],[],[],[],[]
meanPosteriorMeanRecoveryTimeMCMC,lambdaPosteriorMeanRecoveryTimeMCMC,posteriorCOVRecoveryTimeMCMC,lambdaPosteriorAlphaRecoveryTimeMCMC,lambdaPosteriorBetaRecoveryTimeMCMC = [],[],[],[],[]

# Perform Bayesian Parameter Estimation using Conjugate Prior
for i in range(4):
    if i < 3:
        # Perform Bayesian parameter estimation for permit time
        bayesianParameterEstimation = BayesianParameterEstimation(permitTimeData[i],NSamples,1/meanPriorMeanPermitTime[i],coefficientOfVariation)
        bayesianParameterEstimation.mcmcEstimation()
        lambdaPosteriorMeanPermitTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['lambdaMean'])
        meanPosteriorMeanPermitTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['meanMean'])
        posteriorCOVPermitTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['COV'])
        lambdaPosteriorAlphaPermitTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['alpha'])
        lambdaPosteriorBetaPermitTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['beta'])
    
    # Perform Bayesian parameter estimation for repair time
    bayesianParameterEstimation = BayesianParameterEstimation(repairTimeData[i],NSamples,1/meanPriorMeanRepairTime[i],coefficientOfVariation)
    bayesianParameterEstimation.mcmcEstimation()
    lambdaPosteriorMeanRepairTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['lambdaMean'])
    meanPosteriorMeanRepairTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['meanMean'])
    posteriorCOVRepairTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['COV'])
    lambdaPosteriorAlphaRepairTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['alpha'])
    lambdaPosteriorBetaRepairTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['beta'])
    
    # Perform Bayesian parameter estimation for recovery time
    bayesianParameterEstimation = BayesianParameterEstimation(recoveryTimeData[i],NSamples,1/meanPriorMeanRecoveryTime[i],coefficientOfVariation)
    bayesianParameterEstimation.mcmcEstimation()
    lambdaPosteriorMeanRecoveryTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['lambdaMean'])
    meanPosteriorMeanRecoveryTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['meanMean'])
    posteriorCOVRecoveryTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['COV'])
    lambdaPosteriorAlphaRecoveryTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['alpha'])
    lambdaPosteriorBetaRecoveryTimeMCMC.append(bayesianParameterEstimation.posteriorMCMC['beta'])
t1 = time()

C:\Users\henry\Dropbox\WorkingFolder\JournalPapers\Omoya_Morolake\BayesianUpdatingBuildingSpecificRecovery\Production\PythonCode\ForGithub\bayesianParameterEstimation.py:119: RuntimeWarning: overflow encountered in exp
  acceptProposed = np.random.rand() < np.exp(probAccept)


### Print Results from Markov Chain Monte Carlo Method
The cell below prints the results from the Bayesian parameter estimation using the Markov Chain Monte Carlo method.

In [9]:
# Print permit time results
print('lambdaPosteriorMeanPermitTimeMCMC =' + str(lambdaPosteriorMeanPermitTimeMCMC))
print("")
print('meanPosteriorMeanPermitTimeMCMC =' + str(meanPosteriorMeanPermitTimeMCMC))
print("")
print('posteriorCOVPermitTimeMCMC =' + str(posteriorCOVPermitTimeMCMC))
print("")
print('lambdaPosteriorAlphaPermitTimeMCMC =' + str(lambdaPosteriorAlphaPermitTimeMCMC))
print("")
print('lambdaPosteriorBetaPermitTimeMCMC =' + str(lambdaPosteriorBetaPermitTimeMCMC))
print("")

# Print repair time results
print('lambdaPosteriorMeanRepairTimeMCMC =' + str(lambdaPosteriorMeanRepairTimeMCMC))
print("")
print('meanPosteriorMeanRepairTimeMCMC =' + str(meanPosteriorMeanRepairTimeMCMC))
print("")
print('posteriorCOVRepairTimeMCMC =' + str(posteriorCOVRepairTimeMCMC))
print("")
print('lambdaPosteriorAlphaRepairTimeMCMC =' + str(lambdaPosteriorAlphaRepairTimeMCMC))
print("")
print('lambdaPosteriorBetaRepairTimeMCMC =' + str(lambdaPosteriorBetaRepairTimeMCMC))
print("")

# Print reocvery time statistics
print('lambdaPosteriorMeanRecoveryTimeMCMC =' + str(lambdaPosteriorMeanRecoveryTimeMCMC))
print("")
print('meanPosteriorMeanRecoveryTimeMCMC =' + str(meanPosteriorMeanRecoveryTimeMCMC))
print("")
print('posteriorCOVRecoveryTimeMCMC =' + str(posteriorCOVRecoveryTimeMCMC))
print("")
print('posteriorCOVRecoveryTimeMCMC =' + str(posteriorCOVRecoveryTimeMCMC))
print("")
print('lambdaPosteriorAlphaRecoveryTimeMCMC =' + str(lambdaPosteriorAlphaRecoveryTimeMCMC))
print("")
print('lambdaPosteriorBetaRecoveryTimeMCMC =' + str(lambdaPosteriorBetaRecoveryTimeMCMC))
print("")

# Print run time
print('Run time = ' + str(t1 - t0))

lambdaPosteriorMeanPermitTimeMCMC =[0.00957568114981618, 0.010828170375247734, 0.006817927139581647]

meanPosteriorMeanPermitTimeMCMC =[105.8767567901143, 92.51742648732066, 148.63410455383197]

posteriorCOVPermitTimeMCMC =[0.11826667787430838, 0.042417830677915436, 0.11652029906390234]

lambdaPosteriorAlphaPermitTimeMCMC =[73.20213027794168, 558.9609787687457, 75.91909895088895]

lambdaPosteriorBetaPermitTimeMCMC =[7644.587276106923, 51620.9996147163, 11135.217111684682]

lambdaPosteriorMeanRepairTimeMCMC =[0.011886811123524913, 0.008058106436870511, 0.007905663328163838, 0.005015512836440457]

meanPosteriorMeanRepairTimeMCMC =[84.79905543183385, 124.55087553748113, 128.58644893593572, 204.35357628084466]

posteriorCOVRepairTimeMCMC =[0.07644927634610132, 0.05949302425455626, 0.12944956824095077, 0.15957831608880169]

lambdaPosteriorAlphaRepairTimeMCMC =[22.771535873763085, 254.25282902445053, 61.2793090008046, 41.096996167663484]

lambdaPosteriorBetaRepairTimeMCMC =[1915.697627995153

### Compute Marginal Distribution Based on Priors
The cell below computes the marginal distribution parameters based on the priors

In [10]:
# Start time check
t0 = time()

# Define inputs
durationMin = 0.01
durationMax = 3000
numDurations = 500
numlambdas = 500
lambdaFactor = 4

# Initialize lists used to store marginal distribution parameters
permitTimeVectorPrior,permitTimePDFVectorPrior,meanPermitTimePrior,stdPermitTimePrior,covPermitTimePrior = [],[],[],[],[]
repairTimeVectorPrior,repairTimePDFVectorPrior,meanRepairTimePrior,stdRepairTimePrior,covRepairTimePrior = [],[],[],[],[]
recoveryTimeVectorPrior,recoveryTimePDFVectorPrior,meanRecoveryTimePrior,stdRecoveryTimePrior,covRecoveryTimePrior = [],[],[],[],[]

# Compute marginal distribution based on Prior results
for i in range(4):
    if i < 3:
        # Compute marginal distribution for permit time
        bayesianParameterEstimation.marginalDistribution(1/meanPriorMeanPermitTime[i],coefficientOfVariation,durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
        permitTimeVectorPrior.append(bayesianParameterEstimation.marginal['duration'])
        permitTimePDFVectorPrior.append(bayesianParameterEstimation.marginal['pdf'])
        meanPermitTimePrior.append(bayesianParameterEstimation.marginal['mean'])
        stdPermitTimePrior.append(bayesianParameterEstimation.marginal['SD'])
        covPermitTimePrior.append(stdPermitTimePrior[i]/meanPermitTimePrior[i])
    
    # Compute marginal distribution for repair time
    bayesianParameterEstimation.marginalDistribution(1/meanPriorMeanRepairTime[i],coefficientOfVariation,durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
    repairTimeVectorPrior.append(bayesianParameterEstimation.marginal['duration'])
    repairTimePDFVectorPrior.append(bayesianParameterEstimation.marginal['pdf'])
    meanRepairTimePrior.append(bayesianParameterEstimation.marginal['mean'])
    stdRepairTimePrior.append(bayesianParameterEstimation.marginal['SD'])
    covRepairTimePrior.append(stdRepairTimePrior[i]/meanRepairTimePrior[i])

    # Compute marginal distribution for recovery time
    bayesianParameterEstimation.marginalDistribution(1/meanPriorMeanRecoveryTime[i],coefficientOfVariation,durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
    recoveryTimeVectorPrior.append(bayesianParameterEstimation.marginal['duration'])
    recoveryTimePDFVectorPrior.append(bayesianParameterEstimation.marginal['pdf'])
    meanRecoveryTimePrior.append(bayesianParameterEstimation.marginal['mean'])
    stdRecoveryTimePrior.append(bayesianParameterEstimation.marginal['SD'])
    covRecoveryTimePrior.append(stdRecoveryTimePrior[i]/meanRecoveryTimePrior[i])

# End time check
t1 = time()

### Print Marginal Distribution Results Based on Priors
The cell below prints the marginal distribution parameters based on the priors

In [11]:
# Print marginal distribution for permit time
print('meanPermitTimePrior =' + str(meanPermitTimePrior))
print("")
print('stdPermitTimePrior =' + str(stdPermitTimePrior))
print("")
print('covPermitTimePrior =' + str(covPermitTimePrior))
print("")

# Print marginal distribution for repair time
print('meanRepairTimePrior =' + str(meanRepairTimePrior))
print("")
print('stdRepairTimePrior =' + str(stdRepairTimePrior))
print("")
print('covRepairTimePrior =' + str(covRepairTimePrior))
print("")

# Print marginal distribution for recovery time
print('meanRecoveryTimePrior =' + str(meanRecoveryTimePrior))
print("")
print('stdRecoveryTimePrior =' + str(stdRecoveryTimePrior))
print("")
print('covRecoveryTimePrior =' + str(covRecoveryTimePrior))
print("")

# Print run time
print('Run time = ' + str(t1 - t0))

meanPermitTimePrior =[array([60.77249779]), array([163.93836862]), array([394.94321309])]

stdPermitTimePrior =[68.78347795489759, 182.5401200683643, 418.1721963092592]

covPermitTimePrior =[array([1.13181917]), array([1.11346796]), array([1.05881601])]

meanRepairTimePrior =[array([1.75338649]), array([39.19171839]), array([117.79329982]), array([235.34179761])]

stdRepairTimePrior =[4.056689063347205, 44.983455713498294, 131.75122555725287, 259.7391328228564]

covRepairTimePrior =[array([2.31363085]), array([1.14777962]), array([1.11849507]), array([1.10366767])]

meanRecoveryTimePrior =[array([6.10088039]), array([157.16347321]), array([454.79824448]), array([739.05217154])]

stdRecoveryTimePrior =[8.945657060628607, 175.10893393233863, 470.97389822643544, 672.2747274506304]

covRecoveryTimePrior =[array([1.46628953]), array([1.11418341]), array([1.03556666]), array([0.90964448])]

Run time = 438.8295874595642


### Compute Marginal Distribution CP
The cell below computes the marginal distribution parameters based on the CP method

In [12]:
# Start time check
t0 = time()

# Define inputs
durationMin = 0.01
durationMax = 3000
numDurations = 500
numlambdas = 500
lambdaFactor = 4

# Initialize lists used to store marginal distribution parameters
permitTimeVectorCP,permitTimePDFVectorCP,meanPermitTimeCP,stdPermitTimeCP,covPermitTimeCP = [],[],[],[],[]
repairTimeVectorCP,repairTimePDFVectorCP,meanRepairTimeCP,stdRepairTimeCP,covRepairTimeCP = [],[],[],[],[]
recoveryTimeVectorCP,recoveryTimePDFVectorCP,meanRecoveryTimeCP,stdRecoveryTimeCP,covRecoveryTimeCP = [],[],[],[],[]

# Compute marginal distribution based on CP results
for i in range(4):
    if i < 3:
        # Compute marginal distribution for permit time
        bayesianParameterEstimation.marginalDistribution(lambdaPosteriorMeanPermitTimeCP[i],posteriorCOVPermitTimeCP[i],durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
        permitTimeVectorCP.append(bayesianParameterEstimation.marginal['duration'])
        permitTimePDFVectorCP.append(bayesianParameterEstimation.marginal['pdf'])
        meanPermitTimeCP.append(bayesianParameterEstimation.marginal['mean'])
        stdPermitTimeCP.append(bayesianParameterEstimation.marginal['SD'])
        covPermitTimeCP.append(stdPermitTimeCP[i]/meanPermitTimeCP[i])
    
    # Compute marginal distribution for repair time
    bayesianParameterEstimation.marginalDistribution(lambdaPosteriorMeanRepairTimeCP[i],posteriorCOVRepairTimeCP[i],durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
    repairTimeVectorCP.append(bayesianParameterEstimation.marginal['duration'])
    repairTimePDFVectorCP.append(bayesianParameterEstimation.marginal['pdf'])
    meanRepairTimeCP.append(bayesianParameterEstimation.marginal['mean'])
    stdRepairTimeCP.append(bayesianParameterEstimation.marginal['SD'])
    covRepairTimeCP.append(stdRepairTimeCP[i]/meanRepairTimeCP[i])

    # Compute marginal distribution for recovery time
    bayesianParameterEstimation.marginalDistribution(lambdaPosteriorMeanRecoveryTimeCP[i],posteriorCOVRecoveryTimeCP[i],durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
    recoveryTimeVectorCP.append(bayesianParameterEstimation.marginal['duration'])
    recoveryTimePDFVectorCP.append(bayesianParameterEstimation.marginal['pdf'])
    meanRecoveryTimeCP.append(bayesianParameterEstimation.marginal['mean'])
    stdRecoveryTimeCP.append(bayesianParameterEstimation.marginal['SD'])
    covRecoveryTimeCP.append(stdRecoveryTimeCP[i]/meanRecoveryTimeCP[i])

# End time check
t1 = time()

### Print Marginal Distribution Results CP
The cell below prints the marginal distribution parameters based on the CP method

In [13]:
# Print marginal distribution for permit time
print('meanPermitTimeCP =' + str(meanPermitTimeCP))
print("")
print('stdPermitTimeCP =' + str(stdPermitTimeCP))
print("")
print('covPermitTimeCP =' + str(covPermitTimeCP))
print("")

# Print marginal distribution for repair time
print('meanRepairTimeCP =' + str(meanRepairTimeCP))
print("")
print('stdRepairTimeCP =' + str(stdRepairTimeCP))
print("")
print('covRepairTimeCP =' + str(covRepairTimeCP))
print("")

# Print marginal distribution for recovery time
print('meanRecoveryTimeCP =' + str(meanRecoveryTimeCP))
print("")
print('stdRecoveryTimeCP =' + str(stdRecoveryTimeCP))
print("")
print('covRecoveryTimeCP =' + str(covRecoveryTimeCP))
print("")

# Print run time
print('Run time = ' + str(t1 - t0))

meanPermitTimeCP =[array([103.64107471]), array([91.97171881]), array([145.73641241])]

stdPermitTimeCP =[106.63525419917156, 93.83174520217395, 149.22255490211126]

covPermitTimeCP =[array([1.02888989]), array([1.0202239]), array([1.02392087])]

meanRepairTimeCP =[array([83.95782118]), array([123.59202538]), array([125.36688924]), array([196.93327567])]

stdRepairTimeCP =[86.06296365104652, 125.7037240744593, 128.98347056956595, 203.2378803492586]

covRepairTimeCP =[array([1.02507381]), array([1.01708604]), array([1.02884798]), array([1.03201391])]

meanRecoveryTimeCP =[array([165.76586372]), array([229.69200522]), array([269.22175646]), array([487.42384647])]

stdRecoveryTimeCP =[168.52909859368387, 232.32007963305588, 275.0338900354138, 482.12785200362475]

covRecoveryTimeCP =[array([1.0166695]), array([1.01144173]), array([1.02158865]), array([0.98913472])]

Run time = 147.29925274848938


### Compute Marginal Distribution MCMC
The cell below computes the marginal distribution parameters based on the MCMC method

In [14]:
# Start time check
t0 = time()

# Define inputs
durationMin = 0.01
durationMax = 3000
numDurations = 500
numlambdas = 500
lambdaFactor = 4

# Initialize lists used to store marginal distribution parameters
permitTimeVectorMCMC,permitTimePDFVectorMCMC,meanPermitTimeMCMC,stdPermitTimeMCMC,covPermitTimeMCMC = [],[],[],[],[]
repairTimeVectorMCMC,repairTimePDFVectorMCMC,meanRepairTimeMCMC,stdRepairTimeMCMC,covRepairTimeMCMC = [],[],[],[],[]
recoveryTimeVectorMCMC,recoveryTimePDFVectorMCMC,meanRecoveryTimeMCMC,stdRecoveryTimeMCMC,covRecoveryTimeMCMC = [],[],[],[],[]

# Compute marginal distribution based on MCMC results
for i in range(4):
    if i < 3:
        # Compute marginal distribution for permit time
        bayesianParameterEstimation.marginalDistribution(lambdaPosteriorMeanPermitTimeMCMC[i],posteriorCOVPermitTimeMCMC[i],durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
        permitTimeVectorMCMC.append(bayesianParameterEstimation.marginal['duration'])
        permitTimePDFVectorMCMC.append(bayesianParameterEstimation.marginal['pdf'])
        meanPermitTimeMCMC.append(bayesianParameterEstimation.marginal['mean'])
        stdPermitTimeMCMC.append(bayesianParameterEstimation.marginal['SD'])
        covPermitTimeMCMC.append(stdPermitTimeMCMC[i]/meanPermitTimeMCMC[i])
    
    # Compute marginal distribution for repair time
    bayesianParameterEstimation.marginalDistribution(lambdaPosteriorMeanRepairTimeMCMC[i],posteriorCOVRepairTimeMCMC[i],durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
    repairTimeVectorMCMC.append(bayesianParameterEstimation.marginal['duration'])
    repairTimePDFVectorMCMC.append(bayesianParameterEstimation.marginal['pdf'])
    meanRepairTimeMCMC.append(bayesianParameterEstimation.marginal['mean'])
    stdRepairTimeMCMC.append(bayesianParameterEstimation.marginal['SD'])
    covRepairTimeMCMC.append(stdRepairTimeMCMC[i]/meanRepairTimeMCMC[i])

    # Compute marginal distribution for recovery time
    bayesianParameterEstimation.marginalDistribution(lambdaPosteriorMeanRecoveryTimeMCMC[i],posteriorCOVRecoveryTimeMCMC[i],durationMin,durationMax,numDurations,numlambdas,lambdaFactor)
    recoveryTimeVectorMCMC.append(bayesianParameterEstimation.marginal['duration'])
    recoveryTimePDFVectorMCMC.append(bayesianParameterEstimation.marginal['pdf'])
    meanRecoveryTimeMCMC.append(bayesianParameterEstimation.marginal['mean'])
    stdRecoveryTimeMCMC.append(bayesianParameterEstimation.marginal['SD'])
    covRecoveryTimeMCMC.append(stdRecoveryTimeMCMC[i]/meanRecoveryTimeMCMC[i])

# End time check
t1 = time()

### Print Marginal Distribution Results MCMC
The cell below prints the marginal distribution parameters based on the MCMC method

In [15]:
# Print marginal distribution for permit time
print('meanPermitTimeMCMC =' + str(meanPermitTimeMCMC))
print("")
print('stdPermitTimeMCMC =' + str(stdPermitTimeMCMC))
print("")
print('covPermitTimeMCMC =' + str(covPermitTimeMCMC))
print("")

# Print marginal distribution for repair time
print('meanRepairTimeMCMC =' + str(meanRepairTimeMCMC))
print("")
print('stdRepairTimeMCMC =' + str(stdRepairTimeMCMC))
print("")
print('covRepairTimeMCMC =' + str(covRepairTimeMCMC))
print("")

# Print marginal distribution for recovery time
print('meanRecoveryTimeMCMC =' + str(meanRecoveryTimeMCMC))
print("")
print('stdRecoveryTimeMCMC =' + str(stdRecoveryTimeMCMC))
print("")
print('covRecoveryTimeMCMC =' + str(covRecoveryTimeMCMC))
print("")

# Print run time
print('Run time = ' + str(t1 - t0))

meanPermitTimeMCMC =[array([105.46095668]), array([92.11632142]), array([148.07645942])]

stdPermitTimeMCMC =[108.72564502020832, 94.0181682999514, 151.9561818739925]

covPermitTimeMCMC =[array([1.03095637]), array([1.02064614]), array([1.02620081])]

meanRepairTimeMCMC =[array([84.24798684]), array([124.0178448]), array([128.10981262]), array([203.74942216])]

stdRepairTimeMCMC =[86.47174447245543, 126.2427738099307, 132.1066706852242, 211.02405169739953]

covRepairTimeMCMC =[array([1.02639538]), array([1.01794039]), array([1.03119869]), array([1.0357038])]

meanRecoveryTimeMCMC =[array([166.50504809]), array([230.76306572]), array([275.31861429]), array([504.3688547])]

stdRecoveryTimeMCMC =[169.55981744676708, 233.60831872816536, 281.9919698833266, 497.4692982127691]

covRecoveryTimeMCMC =[array([1.01834641]), array([1.01232976]), array([1.02423866]), array([0.98632042])]

Run time = 151.3693642616272
